In [1]:
import numpy as np
import pandas as pd

# Метод query() - функция запроса

    План урока
    - Использование операторов сравнения в query()
    - Использование оператора in в query() (эквивалентно isin())
    - Использование строковых методов в query()
    - Движок (парметр engine) в query
    - Обработка пропущенных значений (NaN или None)
    - Определение условий для индекса (метки-индекса) с помощью query()
    - Определение нескольких условий с помощью query()
    - Обработка имен столбцов с пробелами или точками с помощью query()
    - Обновление исходного объекта с помощью аргумента inplace

# Методы query() и eval()

### - query() - это запрос

### Данная функция запроса используется для извлечения записей на основе указанного выражения и возвращает новый датафрейм. 
### Выражение — это условие или комбинация условий, записанная в виде строки.
### Внутри функции query это выражение оценивается с помощью функции eval(), которая возвращает булеву серию, которая тут же применяется к датафрейму, и в итоге нам возвращается подмножество тех записей, где выражение оценивается как True.

### .eval() возвращает булеву серию (значенияTrue/False)
### .query() возвращает строки датафрейма, соответсвующие условию

### Пример выражения: 'age > 30' (строка с условием)

In [2]:
df = pd.read_csv('files/sample_1.csv')
df

,name,age,state,point
0,Alice,24,NY,64
1,Bob,42,CA,92
2,Charlie,18,CA,70
3,Dave,68,TX,70
4,Ellen,24,CA,88
5,Frank,30,NY,57


In [3]:
# .eval() возвращает булеву серию 
df.eval('age > 30')

0    False
1     True
2    False
3     True
4    False
5    False
dtype: bool

In [4]:
# .query() возвращает строки датафрейма, соответсвующие условию
# внути метода .query() сработал метод .eval() и вадал результат в виде уже отфильтрованных строчек
df.query('age > 30')

,name,age,state,point
1,Bob,42,CA,92
3,Dave,68,TX,70


# Использование операторов сравнения в query()

In [5]:
df[df['age'] < 25]

,name,age,state,point
0,Alice,24,NY,64
2,Charlie,18,CA,70
4,Ellen,24,CA,88


In [6]:
df.query('age < 25')

,name,age,state,point
0,Alice,24,NY,64
2,Charlie,18,CA,70
4,Ellen,24,CA,88


In [7]:
# чтобы использовать переменную в строке запроса, необходимо добавить к имени переменной префикс @
val = 25
df.query('age < @val')

,name,age,state,point
0,Alice,24,NY,64
2,Charlie,18,CA,70
4,Ellen,24,CA,88


In [8]:
# можно использовать цепочку сравнений (Chained comparison)
df.query('30 <= age < 50')

,name,age,state,point
1,Bob,42,CA,92
5,Frank,30,NY,57


### Столбцы можно сравнивать друг с другом, также можно использовать арифметические операторы.

In [9]:
# выводим строчки, где возраст меньше чем результат деления количества баллов на три
df.query('age < point / 3')

,name,age,state,point
2,Charlie,18,CA,70
4,Ellen,24,CA,88


## Использование кавычек в выражении 
- строки внутри строки запроса должны быть заключены в кавычки
- Двойные кавычки (") могут использоваться внутри строк с одинарными кавычками (') и наоборот. Тот же тип кавычек может быть использован, если он экранирован обратной косой чертой (\\).

### Использование кавычек. Пример 1

In [10]:
# строку внутри строки заключаем в кавычки
# используем двойные кавычки внутри одинарных
df.query('state == "CA"')

,name,age,state,point
1,Bob,42,CA,92
2,Charlie,18,CA,70
4,Ellen,24,CA,88


### Использование кавычек. Пример 2

In [11]:
# строку внутри строки заключаем в кавычки
# используем одинарные кавычки внутри двойных
df.query("state == 'CA'")

,name,age,state,point
1,Bob,42,CA,92
2,Charlie,18,CA,70
4,Ellen,24,CA,88


### Использование кавычек. Пример 3

In [12]:
# Мы можем использовать тот же тип кавычек, если они экранированы обратной косой чертой \.
df.query("state == \"CA\"")

,name,age,state,point
1,Bob,42,CA,92
2,Charlie,18,CA,70
4,Ellen,24,CA,88


### Использование кавычек. Пример 4

In [13]:
df.query('state == \'CA\'')

,name,age,state,point
1,Bob,42,CA,92
2,Charlie,18,CA,70
4,Ellen,24,CA,88


In [14]:
# переменные не требуют кавычек
s = 'CA'
df.query('state != @s')

,name,age,state,point
0,Alice,24,NY,64
3,Dave,68,TX,70
5,Frank,30,NY,57


# Использование оператора in с query() (эквивалентно isin())

In [15]:
df[df['state'].isin(['NY', 'TX'])]

,name,age,state,point
0,Alice,24,NY,64
3,Dave,68,TX,70
5,Frank,30,NY,57


In [16]:
df.query('state in ["NY", "TX"]')

,name,age,state,point
0,Alice,24,NY,64
3,Dave,68,TX,70
5,Frank,30,NY,57


In [17]:
states=["NY", "TX"]
df.query('state in @states')

,name,age,state,point
0,Alice,24,NY,64
3,Dave,68,TX,70
5,Frank,30,NY,57


In [18]:
states=['NY', 'TX']
df.query('state in @states')

,name,age,state,point
0,Alice,24,NY,64
3,Dave,68,TX,70
5,Frank,30,NY,57


In [19]:
df.query('state == ["NY", "TX"]')

,name,age,state,point
0,Alice,24,NY,64
3,Dave,68,TX,70
5,Frank,30,NY,57


# Использование строковых методов с query()

- str.contains(): Проверяет, содержится ли определенная подстрока в строке (соответсвует ли шаблону регулярного выражения)
- str.endswith(): Проверяет, заканчивается ли строка определенной подстрокой
- str.startswith(): Проверяет, начинается ли строка с определенной подстроки
- str.match(): Проверяет, соответствует ли строка шаблону регулярного выражения (с начала сроки)

In [20]:
# найдем все строки, где имя заканчивается на 'e'
df.query('name.str.endswith("e")')

,name,age,state,point
0,Alice,24,NY,64
2,Charlie,18,CA,70
3,Dave,68,TX,70


In [21]:
# если предыдущий код упал с ошибкой, укажите напрямую параметр engine='python'
df.query('name.str.endswith("e")', engine='python')

,name,age,state,point
0,Alice,24,NY,64
2,Charlie,18,CA,70
3,Dave,68,TX,70


### Движок engine
### параметр engine в методе query: 'python', 'numexpr' (по-умолчанию стоит 'numexpr')
### NumExpr — это быстрый пакет для вычисления числовых выражений для NumPy

### Используя строковые методы, имейте в виду, что если у вас более рання версия pandas (например, версия pandas 1.3.4), то код может упасть с ошибкой. Чтобы этого не произошло, вам необходимо либо обновить pandas до последней версии или прописать параметр engine='python'
### Проверить, какая у вас версия: 
pd.show_versions(as_json=False)
### Обновить версию:
pip3 install --upgrade pandas

In [22]:
df.query('name.str.slice(2,5,2)=="al"')

,name,age,state,point
2,Charlie,18,CA,70


In [23]:
df.query('name.str.get(2)=="a"')

,name,age,state,point
2,Charlie,18,CA,70
5,Frank,30,NY,57


In [24]:
df.query('name.str.get(-2)=="n"')

,name,age,state,point
5,Frank,30,NY,57


In [25]:
# найдем записи, в которых возраст заканчивается на 8
df.query('age.astype("str").str.endswith("8")')

,name,age,state,point
2,Charlie,18,CA,70
3,Dave,68,TX,70


In [26]:
# такого же результата можно добиться, не прибегая к строковым методам
# находим остаток от деления возраста на 10 равный 8
df.query('age % 10 == 8')

,name,age,state,point
2,Charlie,18,CA,70
3,Dave,68,TX,70


# Обработка пропущенных значений (NaN или None)

In [27]:
df2 = df.copy()
df2.loc[0, 'name'] = None
df2

,name,age,state,point
0,None,24,NY,64
1,Bob,42,CA,92
2,Charlie,18,CA,70
3,Dave,68,TX,70
4,Ellen,24,CA,88
5,Frank,30,NY,57


In [28]:
# ValueError: Cannot mask with non-boolean array containing NA / NaN values
# df2.query('name.str.endswith("e")')

In [29]:
# В query() можно указать аргумент na для строковых методов.
df2.query('name.str.endswith("e", na=False)')

,name,age,state,point
2,Charlie,18,CA,70
3,Dave,68,TX,70


# Определение условий для индекса с помощью query()

In [30]:
# выводим строки только с четными индексами
df.query('index % 2 == 0')

,name,age,state,point
0,Alice,24,NY,64
2,Charlie,18,CA,70
4,Ellen,24,CA,88


In [31]:
df_name = df.set_index('name')
df_name

,age,state,point
name,,,
Alice,24,NY,64
Bob,42,CA,92
Charlie,18,CA,70
Dave,68,TX,70
Ellen,24,CA,88
Frank,30,NY,57


In [32]:
df_name.query('name.str.endswith("e")')

,age,state,point
name,,,
Alice,24,NY,64
Charlie,18,CA,70
Dave,68,TX,70


In [33]:
df_name.query('index.str.endswith("e")')

,age,state,point
name,,,
Alice,24,NY,64
Charlie,18,CA,70
Dave,68,TX,70


# Определение нескольких условий с помощью query()

In [34]:
# определение нескольких условий с помощью булевой индексации
df[(df['age'] < 25) & (df['point'] > 65)]

,name,age,state,point
2,Charlie,18,CA,70
4,Ellen,24,CA,88


В методе query:

- Скобки вокруг каждого условия не требуются 
- логическое И можно представить в виде & или and
- логическое ИЛИ может быть представлено в виде | или or
- отрицание представлено в виде not, также для отрицания можно использовать ~ 
- В случае трех и более условий безопаснее явно заключать главное условие в круглые скобки, поскольку результат может меняться в зависимости от порядка следования. Например, & имеет больший приоритет, чем |.

### Логическое И

In [35]:
df.query('age < 25 & point > 65')

,name,age,state,point
2,Charlie,18,CA,70
4,Ellen,24,CA,88


In [36]:
df.query('age < 25 and point > 65')

,name,age,state,point
2,Charlie,18,CA,70
4,Ellen,24,CA,88


### Логическое ИЛИ

In [37]:
df.query('age < 20 | point > 80')

,name,age,state,point
1,Bob,42,CA,92
2,Charlie,18,CA,70
4,Ellen,24,CA,88


In [38]:
df.query('age < 20 or point > 80')

,name,age,state,point
1,Bob,42,CA,92
2,Charlie,18,CA,70
4,Ellen,24,CA,88


### Логическое ОТРИЦАНИЕ

In [39]:
df.query('not age < 25 and not point > 65')

,name,age,state,point
5,Frank,30,NY,57


In [40]:
df.query('~ (age < 25) and ~ (point > 65)')

,name,age,state,point
5,Frank,30,NY,57


###  ТРИ и более условий в query

In [41]:
df.query('age == 24 | point > 80 & state == "CA"')

,name,age,state,point
0,Alice,24,NY,64
1,Bob,42,CA,92
4,Ellen,24,CA,88


In [42]:
# безопаснее явно заключать главное условие в круглые скобки
# результат может меняться в зависимости от порядка следования. Например, & имеет больший приоритет, чем |.
df.query('(age == 24 | point > 80) & state == "CA"')

,name,age,state,point
1,Bob,42,CA,92
4,Ellen,24,CA,88


#  Обработка имен столбцов с помощью query()

In [43]:
df2 = df.copy()
df2.columns = ['0name', 'age.year', 'state name', 'points(out_of_100)']
df2

,0name,age.year,state name,points(out_of_100)
0,Alice,24,NY,64
1,Bob,42,CA,92
2,Charlie,18,CA,70
3,Dave,68,TX,70
4,Ellen,24,CA,88
5,Frank,30,NY,57


In [44]:
# df2.query('0name.str.endswith("e")')
# SyntaxError: invalid syntax

In [45]:
# df2.query('age.year < 25')
# UndefinedVariableError: name 'age' is not defined

In [46]:
# df2.query('state name == "CA"')
# SyntaxError: invalid syntax

In [47]:
# df2.query('points(out_of_100) > 40')
# ValueError: "points" is not a supported function

### - Используем бэктик (обратный апостроф)

In [48]:
# заключаем в бэктики проблемные имена
df2.query('`0name`.str.endswith("e")')

,0name,age.year,state name,points(out_of_100)
0,Alice,24,NY,64
2,Charlie,18,CA,70
3,Dave,68,TX,70


In [49]:
df2.query('`age.year` < 25')

,0name,age.year,state name,points(out_of_100)
0,Alice,24,NY,64
2,Charlie,18,CA,70
4,Ellen,24,CA,88


In [50]:
df2.query('`state name` == "CA"')

,0name,age.year,state name,points(out_of_100)
1,Bob,42,CA,92
2,Charlie,18,CA,70
4,Ellen,24,CA,88


In [51]:
df2.query('`points(out_of_100)` > 40')

,0name,age.year,state name,points(out_of_100)
0,Alice,24,NY,64
1,Bob,42,CA,92
2,Charlie,18,CA,70
3,Dave,68,TX,70
4,Ellen,24,CA,88
5,Frank,30,NY,57


In [52]:
df3 =pd.read_csv('files/sample_1.csv', skiprows=1, header=None)
df3

,0,1,2,3
0,Alice,24,NY,64
1,Bob,42,CA,92
2,Charlie,18,CA,70
3,Dave,68,TX,70
4,Ellen,24,CA,88
5,Frank,30,NY,57


In [53]:
# df3.query('3 > 75')

In [54]:
# бэктики не помогут
# df3.query('`3`> 75')
# UndefinedVariableError: name 'BACKTICK_QUOTED_STRING_3' is not defined

In [55]:
# лучше прибегать к использованию булевой индексации
df3[df3[3] > 75]

,0,1,2,3
1,Bob,42,CA,92
4,Ellen,24,CA,88


In [56]:
# можно дать колонкам названия через атрибут columns
df3.columns=['col_1', 'col_2', 'col_3', 'col_4']
df3

,col_1,col_2,col_3,col_4
0,Alice,24,NY,64
1,Bob,42,CA,92
2,Charlie,18,CA,70
3,Dave,68,TX,70
4,Ellen,24,CA,88
5,Frank,30,NY,57


In [57]:
df3.query('col_3 == "CA"')

,col_1,col_2,col_3,col_4
1,Bob,42,CA,92
2,Charlie,18,CA,70
4,Ellen,24,CA,88


# Обновление исходного объекта с помощью аргумента inplace

In [58]:
df4 = df.copy()
df4.query('age > 25', inplace=True)
df4

,name,age,state,point
1,Bob,42,CA,92
3,Dave,68,TX,70
5,Frank,30,NY,57
